In [1]:
import os
import torch
import torchvision
import torch.nn as nn
from torchvision import transforms
from torchvision.utils import save_image

In [3]:
#Device configuration
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [20]:
#Hyper-parameters
latent_size = 64
hidden_size = 256 
image_size = 784
num_epochs = 200
batch_size = 100
sample_dir = 'samples'

In [22]:
# Create a directory if not exists
if not os.path.exists(sample_dir):
    os.makedirs(sample_dir)

In [23]:
# Image processing
# transform = transforms.Compose([
#                 transforms.ToTensor(),
#                 transforms.Normalize(mean=(0.5, 0.5, 0.5),   # 3 for RGB channels
#                                      std=(0.5, 0.5, 0.5))])
transform = transforms.Compose([
                transforms.ToTensor(),
                transforms.Normalize(mean=[0.5],   # 1 for greyscale channels
                                     std=[0.5])])

In [24]:
# MNIST dataset
mnist = torchvision.datasets.MNIST(root='../../data/',
                                   train=True,
                                   transform=transform,
                                   download=True)

In [25]:
# Data loader #torch model로 실행 할 수 있게 데이터 변경
data_loader = torch.utils.data.DataLoader(dataset=mnist,
                                          batch_size=batch_size, 
                                          shuffle=True)

In [26]:
# Discriminator
D = nn.Sequential(
    nn.Linear(image_size, hidden_size),
    nn.LeakyReLU(0.2),
    nn.Linear(hidden_size, hidden_size),
    nn.LeakyReLU(0.2),
    nn.Linear(hidden_size, 1),
    nn.Sigmoid())

# Generator 
G = nn.Sequential(
    nn.Linear(latent_size, hidden_size),
    nn.ReLU(),
    nn.Linear(hidden_size, hidden_size),
    nn.ReLU(),
    nn.Linear(hidden_size, image_size),
    nn.Tanh())

In [27]:
# Device setting
D = D.to(device)
G = G.to(device)

In [28]:
# Binary cross entropy loss and optimizer
criterion = nn.BCELoss()
d_optimizer = torch.optim.Adam(D.parameters(), lr=0.0002)
g_optimizer = torch.optim.Adam(G.parameters(), lr=0.0002)

In [29]:
#normalization
def denorm(x):
    out = (x + 1) / 2
    return out.clamp(0, 1)
#가중치 초기화
def reset_grad():
    d_optimizer.zero_grad()
    g_optimizer.zero_grad()

In [30]:
# Start training
total_step = len(data_loader)

for epoch in range(num_epochs):
    for i, (images, _) in enumerate(data_loader):
        images = images.reshape(batch_size, -1).to(device)
        
        # Create the labels which are later used as input for the BCE loss
        real_labels = torch.ones(batch_size, 1).to(device)
        fake_labels = torch.zeros(batch_size, 1).to(device)

        # ================================================================== #
        #                      Train the discriminator                       #
        # ================================================================== #

        # Compute BCE_Loss using real images where BCE_Loss(x, y): - y * log(D(x)) - (1-y) * log(1 - D(x))
        # Second term of the loss is always zero since real_labels == 1
        outputs = D(images)
        d_loss_real = criterion(outputs, real_labels)
        real_score = outputs
        
        # Compute BCELoss using fake images
        # First term of the loss is always zero since fake_labels == 0
        z = torch.randn(batch_size, latent_size).to(device)
        fake_images = G(z)
        outputs = D(fake_images)
        d_loss_fake = criterion(outputs, fake_labels)
        fake_score = outputs
        
        # Backprop and optimize
        d_loss = d_loss_real + d_loss_fake
        reset_grad()
        d_loss.backward()
        d_optimizer.step()
        
        # ================================================================== #
        #                        Train the generator                         #
        # ================================================================== #

        # Compute loss with fake images
        z = torch.randn(batch_size, latent_size).to(device)
        fake_images = G(z)
        outputs = D(fake_images)
        
        # We train G to maximize log(D(G(z)) instead of minimizing log(1-D(G(z)))
        # For the reason, see the last paragraph of section 3. https://arxiv.org/pdf/1406.2661.pdf
        g_loss = criterion(outputs, real_labels)
        
        # Backprop and optimize
        reset_grad()
        g_loss.backward()
        g_optimizer.step()
        
        if (i+1) % 200 == 0:
            print('Epoch [{}/{}], Step [{}/{}], d_loss: {:.4f}, g_loss: {:.4f}, D(x): {:.2f}, D(G(z)): {:.2f}' 
                  .format(epoch, num_epochs, i+1, total_step, d_loss.item(), g_loss.item(), 
                          real_score.mean().item(), fake_score.mean().item()))
                # Save real images
    if (epoch+1) == 1:
        images = images.reshape(images.size(0), 1, 28, 28)
        save_image(denorm(images), os.path.join(sample_dir, 'real_images.png'))
    
    # Save sampled images
    fake_images = fake_images.reshape(fake_images.size(0), 1, 28, 28)
    save_image(denorm(fake_images), os.path.join(sample_dir, 'fake_images-{}.png'.format(epoch+1)))

# Save the model checkpoints 
torch.save(G.state_dict(), 'G.ckpt')
torch.save(D.state_dict(), 'D.ckpt')
    

Epoch [0/200], Step [200/600], d_loss: 0.0425, g_loss: 4.1751, D(x): 0.99, D(G(z)): 0.04
Epoch [0/200], Step [400/600], d_loss: 0.0283, g_loss: 6.1857, D(x): 0.99, D(G(z)): 0.02
Epoch [0/200], Step [600/600], d_loss: 0.1247, g_loss: 3.8669, D(x): 0.96, D(G(z)): 0.08
Epoch [1/200], Step [200/600], d_loss: 0.0682, g_loss: 5.6731, D(x): 0.99, D(G(z)): 0.06
Epoch [1/200], Step [400/600], d_loss: 0.2587, g_loss: 4.1959, D(x): 0.91, D(G(z)): 0.08
Epoch [1/200], Step [600/600], d_loss: 0.1596, g_loss: 4.9260, D(x): 0.95, D(G(z)): 0.07
Epoch [2/200], Step [200/600], d_loss: 0.2183, g_loss: 4.3602, D(x): 0.95, D(G(z)): 0.14
Epoch [2/200], Step [400/600], d_loss: 0.6020, g_loss: 3.2751, D(x): 0.75, D(G(z)): 0.14
Epoch [2/200], Step [600/600], d_loss: 0.3517, g_loss: 3.5961, D(x): 0.85, D(G(z)): 0.07
Epoch [3/200], Step [200/600], d_loss: 0.9139, g_loss: 2.5043, D(x): 0.80, D(G(z)): 0.37
Epoch [3/200], Step [400/600], d_loss: 0.1955, g_loss: 3.5754, D(x): 0.93, D(G(z)): 0.09
Epoch [3/200], Step [

Epoch [30/200], Step [600/600], d_loss: 0.2902, g_loss: 4.4282, D(x): 0.91, D(G(z)): 0.10
Epoch [31/200], Step [200/600], d_loss: 0.3965, g_loss: 3.9841, D(x): 0.93, D(G(z)): 0.18
Epoch [31/200], Step [400/600], d_loss: 0.3153, g_loss: 4.1557, D(x): 0.88, D(G(z)): 0.03
Epoch [31/200], Step [600/600], d_loss: 0.2835, g_loss: 3.0614, D(x): 0.90, D(G(z)): 0.07
Epoch [32/200], Step [200/600], d_loss: 0.3500, g_loss: 3.4260, D(x): 0.87, D(G(z)): 0.08
Epoch [32/200], Step [400/600], d_loss: 0.3311, g_loss: 4.7278, D(x): 0.90, D(G(z)): 0.06
Epoch [32/200], Step [600/600], d_loss: 0.3312, g_loss: 3.9567, D(x): 0.86, D(G(z)): 0.06
Epoch [33/200], Step [200/600], d_loss: 0.3120, g_loss: 3.0853, D(x): 0.90, D(G(z)): 0.11
Epoch [33/200], Step [400/600], d_loss: 0.3565, g_loss: 4.1516, D(x): 0.90, D(G(z)): 0.10
Epoch [33/200], Step [600/600], d_loss: 0.3891, g_loss: 2.9209, D(x): 0.89, D(G(z)): 0.13
Epoch [34/200], Step [200/600], d_loss: 0.3961, g_loss: 4.9456, D(x): 0.93, D(G(z)): 0.19
Epoch [34/

Epoch [61/200], Step [400/600], d_loss: 0.4469, g_loss: 2.7247, D(x): 0.79, D(G(z)): 0.11
Epoch [61/200], Step [600/600], d_loss: 0.6507, g_loss: 1.9727, D(x): 0.78, D(G(z)): 0.21
Epoch [62/200], Step [200/600], d_loss: 0.8553, g_loss: 2.8177, D(x): 0.74, D(G(z)): 0.21
Epoch [62/200], Step [400/600], d_loss: 0.7982, g_loss: 2.1997, D(x): 0.72, D(G(z)): 0.22
Epoch [62/200], Step [600/600], d_loss: 0.7820, g_loss: 2.2757, D(x): 0.77, D(G(z)): 0.25
Epoch [63/200], Step [200/600], d_loss: 0.8010, g_loss: 2.3688, D(x): 0.78, D(G(z)): 0.28
Epoch [63/200], Step [400/600], d_loss: 0.6818, g_loss: 2.1351, D(x): 0.81, D(G(z)): 0.27
Epoch [63/200], Step [600/600], d_loss: 0.6860, g_loss: 2.0799, D(x): 0.76, D(G(z)): 0.19
Epoch [64/200], Step [200/600], d_loss: 0.7631, g_loss: 2.0387, D(x): 0.81, D(G(z)): 0.27
Epoch [64/200], Step [400/600], d_loss: 0.6504, g_loss: 1.9246, D(x): 0.86, D(G(z)): 0.31
Epoch [64/200], Step [600/600], d_loss: 0.5544, g_loss: 3.3457, D(x): 0.76, D(G(z)): 0.11
Epoch [65/

Epoch [92/200], Step [200/600], d_loss: 0.7765, g_loss: 1.7644, D(x): 0.76, D(G(z)): 0.26
Epoch [92/200], Step [400/600], d_loss: 0.8026, g_loss: 2.1336, D(x): 0.77, D(G(z)): 0.31
Epoch [92/200], Step [600/600], d_loss: 0.7834, g_loss: 1.5846, D(x): 0.72, D(G(z)): 0.21
Epoch [93/200], Step [200/600], d_loss: 0.8244, g_loss: 1.6284, D(x): 0.80, D(G(z)): 0.36
Epoch [93/200], Step [400/600], d_loss: 0.6168, g_loss: 2.3577, D(x): 0.83, D(G(z)): 0.23
Epoch [93/200], Step [600/600], d_loss: 0.9077, g_loss: 1.7094, D(x): 0.68, D(G(z)): 0.26
Epoch [94/200], Step [200/600], d_loss: 0.7516, g_loss: 2.1182, D(x): 0.78, D(G(z)): 0.25
Epoch [94/200], Step [400/600], d_loss: 0.6837, g_loss: 2.5228, D(x): 0.78, D(G(z)): 0.19
Epoch [94/200], Step [600/600], d_loss: 0.7447, g_loss: 2.3158, D(x): 0.74, D(G(z)): 0.19
Epoch [95/200], Step [200/600], d_loss: 0.7642, g_loss: 1.8169, D(x): 0.77, D(G(z)): 0.25
Epoch [95/200], Step [400/600], d_loss: 0.9914, g_loss: 1.7706, D(x): 0.72, D(G(z)): 0.31
Epoch [95/

Epoch [122/200], Step [400/600], d_loss: 0.6622, g_loss: 2.3599, D(x): 0.73, D(G(z)): 0.18
Epoch [122/200], Step [600/600], d_loss: 0.9368, g_loss: 1.3649, D(x): 0.77, D(G(z)): 0.34
Epoch [123/200], Step [200/600], d_loss: 0.7820, g_loss: 1.9233, D(x): 0.71, D(G(z)): 0.22
Epoch [123/200], Step [400/600], d_loss: 0.7095, g_loss: 1.6992, D(x): 0.86, D(G(z)): 0.35
Epoch [123/200], Step [600/600], d_loss: 0.9064, g_loss: 1.4886, D(x): 0.66, D(G(z)): 0.26
Epoch [124/200], Step [200/600], d_loss: 0.6378, g_loss: 1.3608, D(x): 0.87, D(G(z)): 0.32
Epoch [124/200], Step [400/600], d_loss: 0.7389, g_loss: 1.8212, D(x): 0.72, D(G(z)): 0.24
Epoch [124/200], Step [600/600], d_loss: 0.9127, g_loss: 1.5778, D(x): 0.69, D(G(z)): 0.30
Epoch [125/200], Step [200/600], d_loss: 0.7962, g_loss: 1.2767, D(x): 0.74, D(G(z)): 0.29
Epoch [125/200], Step [400/600], d_loss: 0.8623, g_loss: 1.6459, D(x): 0.72, D(G(z)): 0.28
Epoch [125/200], Step [600/600], d_loss: 0.7318, g_loss: 2.2428, D(x): 0.71, D(G(z)): 0.21

Epoch [152/200], Step [600/600], d_loss: 0.8818, g_loss: 1.9088, D(x): 0.73, D(G(z)): 0.28
Epoch [153/200], Step [200/600], d_loss: 0.8417, g_loss: 1.7097, D(x): 0.77, D(G(z)): 0.32
Epoch [153/200], Step [400/600], d_loss: 0.8847, g_loss: 1.5573, D(x): 0.80, D(G(z)): 0.37
Epoch [153/200], Step [600/600], d_loss: 0.9840, g_loss: 1.5236, D(x): 0.69, D(G(z)): 0.35
Epoch [154/200], Step [200/600], d_loss: 0.7308, g_loss: 1.9547, D(x): 0.70, D(G(z)): 0.21
Epoch [154/200], Step [400/600], d_loss: 1.0774, g_loss: 1.4951, D(x): 0.56, D(G(z)): 0.22
Epoch [154/200], Step [600/600], d_loss: 0.8920, g_loss: 1.3392, D(x): 0.79, D(G(z)): 0.39
Epoch [155/200], Step [200/600], d_loss: 0.9673, g_loss: 1.8517, D(x): 0.59, D(G(z)): 0.18
Epoch [155/200], Step [400/600], d_loss: 0.9339, g_loss: 1.8305, D(x): 0.62, D(G(z)): 0.21
Epoch [155/200], Step [600/600], d_loss: 0.7169, g_loss: 2.1069, D(x): 0.75, D(G(z)): 0.24
Epoch [156/200], Step [200/600], d_loss: 1.0019, g_loss: 1.8216, D(x): 0.72, D(G(z)): 0.31

Epoch [183/200], Step [200/600], d_loss: 0.8253, g_loss: 1.5360, D(x): 0.74, D(G(z)): 0.31
Epoch [183/200], Step [400/600], d_loss: 0.8765, g_loss: 1.1595, D(x): 0.70, D(G(z)): 0.31
Epoch [183/200], Step [600/600], d_loss: 0.9664, g_loss: 1.6972, D(x): 0.63, D(G(z)): 0.26
Epoch [184/200], Step [200/600], d_loss: 0.7475, g_loss: 1.8245, D(x): 0.72, D(G(z)): 0.24
Epoch [184/200], Step [400/600], d_loss: 1.0333, g_loss: 1.7786, D(x): 0.61, D(G(z)): 0.26
Epoch [184/200], Step [600/600], d_loss: 1.0277, g_loss: 1.6559, D(x): 0.61, D(G(z)): 0.23
Epoch [185/200], Step [200/600], d_loss: 0.8548, g_loss: 1.4126, D(x): 0.76, D(G(z)): 0.33
Epoch [185/200], Step [400/600], d_loss: 1.1028, g_loss: 1.5427, D(x): 0.62, D(G(z)): 0.31
Epoch [185/200], Step [600/600], d_loss: 0.8199, g_loss: 1.2523, D(x): 0.75, D(G(z)): 0.33
Epoch [186/200], Step [200/600], d_loss: 1.0856, g_loss: 1.6038, D(x): 0.60, D(G(z)): 0.26
Epoch [186/200], Step [400/600], d_loss: 0.9887, g_loss: 1.5282, D(x): 0.61, D(G(z)): 0.25